In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 6.4e-07}
band = [3]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.8,), (1.66, 1.8)]
  wgt = [(0.9,), (0.5, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.000668                    1  0.013494
0.000750        2  0.000993                    2  0.013760
0.001052        3  0.001702                    3  0.014332
0.001476        4  0.002457                    4  0.014941
0.002070        5  0.003216                    5  0.015548
0.002904        6  0.003977                    6  0.016166
0.004074        7  0.004733                    7  0.016781
0.005714        8  0.005491                    8  0.017408
0.008015        9  0.006199                    9  0.018030
0.011243       10  0.006888                   10  0.018664
0.015771       11  0.007514                   11  0.019296
0.022122       12  0.008076                   12  0.019925
0.031031       13  0.008559                   13  0.020561
0.043528       14  0.008976                   14  0.021236
0.061057       15  0.010988                   15  0.023288
0.085645       16  0.017475                   16  0.029026
0.120136       17  0.027476                   17  0.037721
0.168516       18  0.038391                   18  0.047372
0.236378       19  0.050029                   19  0.057825
0.331549       20  0.062389                   20  0.069084
0.465100       21  0.075470                   21  0.081189
0.652400       22  0.089347                   22  0.094049
0.915100       23  0.100439                   23  0.104671
1.283650       24  0.099062                   24  0.105192
1.800600       25  0.086465                   25  0.096158
2.525700       26  0.072291                   26  0.084712
3.542800       27  0.059499                   27  0.072965
4.969550       28  0.048743                   28  0.061484
6.970850       29  0.040211                   29  0.050617
9.778100       30  0.033328                   30  0.040345
13.715850      31  0.027008                   31  0.030262
19.239350      32  0.020792                   32  0.020661
26.987250      33  0.015394                   33  0.012610
37.855300      34  0.010745                   34  0.006320
53.100050      35  0.005621                   35  0.000830
73.887500      36  0.000029                   36 -0.004055
97.662500      37 -0.004279                   37 -0.007395
121.437500     38 -0.007235                   38 -0.009540
145.212500     39 -0.010201                   39 -0.011612
168.987500     40 -0.012481                   40 -0.013041
192.762500     41 -0.012027                   41 -0.012240
216.537500     42 -0.009212                   42 -0.009485
240.312500     43 -0.006214                   43 -0.006461
264.087500     44 -0.003444                   44 -0.003570
287.862500     45 -0.000840                   45 -0.000805
311.637500     46  0.001645                   46  0.001854
335.412500     47  0.004040                   47  0.004427
359.187500     48  0.006359                   48  0.006916
382.962500     49  0.008612                   49  0.009333
406.737500     50  0.010792                   50  0.011674
430.512500     51  0.012891                   51  0.013925
454.287500     52  0.014896                   52  0.016082
478.062500     53  0.016806                   53  0.018151
501.837500     54  0.018636                   54  0.020136
525.612500     55  0.020405                   55  0.022047
549.387500     56  0.022132                   56  0.023914
573.162500     57  0.023828                   57  0.025750
596.937500     58  0.025500                   58  0.027560
620.712500     59  0.027153                   59  0.029350
644.487500     60  0.028791                   60  0.031117
668.262500     61  0.030411                   61  0.032867
692.037500     62  0.032019                   62  0.034601
715.812500     63  0.033621                   63  0.036329
739.587500     64  0.035216                   64  0.038043
763.362500     65  0.036804                   65  0.039738
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -34.292180  0.000000 -34.292180 -34.278444  4.361355e-08   
0.000624    2     -34.292182  0.000003 -34.292180 -34.278445  2.242855e-06   
0.000876    3     -34.292183  0.000004 -34.292180 -34.278445  3.133049e-06   
0.001229    4     -34.292185  0.000005 -34.292180 -34.278446  4.393768e-06   
0.001723    5     -34.292187  0.000007 -34.292179 -34.278447  6.180662e-06   
0.002417    6     -34.292190  0.000010 -34.292179 -34.278448  8.712491e-06   
0.003391    7     -34.292194  0.000015 -34.292179 -34.278450  1.230052e-05   
0.004757    8     -34.292199  0.000021 -34.292178 -34.278452  1.738479e-05   
0.006672    9     -34.292206  0.000030 -34.292177 -34.278456  2.458941e-05   
0.009359    10    -34.292217  0.000042 -34.292175 -34.278460  3.479721e-05   
0.013128    11    -34.292231  0.000059 -34.292172 -34.278466  4.926148e-05   
0.018415    12    -34.292251  0.000084 -34.292167 -34.278475  6.975332e-05   
0.025830    13    -34.292279  0.000119 -34.292160 -34.278486  9.877992e-05   
0.036232    14    -34.292317  0.000168 -34.292149 -34.278502  1.398974e-04   
0.050823    15    -34.292371  0.000237 -34.292134 -34.278524  1.981735e-04   
0.071291    16    -34.292444  0.000337 -34.292107 -34.278551  2.824809e-04   
0.100000    17    -34.292535  0.000487 -34.292048 -34.278581  4.107832e-04   
0.140271    18    -34.292638  0.000721 -34.291917 -34.278602  6.120690e-04   
0.196760    19    -34.292744  0.001084 -34.291660 -34.278601  9.275333e-04   
0.275997    20    -34.292834  0.001644 -34.291190 -34.278551  1.420166e-03   
0.387100    21    -34.292871  0.002502 -34.290369 -34.278407  2.186164e-03   
0.543100    22    -34.292783  0.003809 -34.288974 -34.278095  3.374645e-03   
0.761700    23    -34.292441  0.005780 -34.286661 -34.277492  5.206774e-03   
1.068500    24    -34.291681  0.008670 -34.283011 -34.276448  7.967001e-03   
1.498800    25    -34.290504  0.012542 -34.277961 -34.274952  1.183351e-02   
2.102400    26    -34.289077  0.017298 -34.271779 -34.273134  1.689170e-02   
2.949000    27    -34.287494  0.022964 -34.264529 -34.271081  2.333504e-02   
4.136600    28    -34.285838  0.029679 -34.256159 -34.268898  3.141813e-02   
5.802500    29    -34.284246  0.037706 -34.246540 -34.266762  4.141633e-02   
8.139200    30    -34.282925  0.047515 -34.235410 -34.264956  5.362281e-02   
11.417000   31    -34.282260  0.059791 -34.222469 -34.263973  6.830587e-02   
16.014700   32    -34.283070  0.075310 -34.207760 -34.264780  8.559659e-02   
22.464000   33    -34.286850  0.094975 -34.191876 -34.269026  1.056283e-01   
31.510500   34    -34.295660  0.120280 -34.175379 -34.279059  1.291756e-01   
44.200100   35    -34.312517  0.153289 -34.159229 -34.298387  1.580046e-01   
62.000000   36    -34.343206  0.195830 -34.147376 -34.333191  1.945596e-01   
85.775000   37    -34.394933  0.247638 -34.147295 -34.390392  2.403385e-01   
109.550000  38    -34.455987  0.296642 -34.159346 -34.456444  2.855625e-01   
133.325000  39    -34.524443  0.344720 -34.179723 -34.529323  3.315707e-01   
157.100000  40    -34.600375  0.391922 -34.208453 -34.608661  3.782019e-01   
180.875000  41    -34.683133  0.439530 -34.243603 -34.693387  4.261967e-01   
204.650000  42    -34.768946  0.491470 -34.277476 -34.780168  4.785019e-01   
228.425000  43    -34.853946  0.550526 -34.303420 -34.865879  5.374970e-01   
252.200000  44    -34.937057  0.616136 -34.320922 -34.949671  6.030918e-01   
275.975000  45    -35.018019  0.687398 -34.330620 -35.031308  6.746729e-01   
299.750000  46    -35.096675  0.763690 -34.332985 -35.110627  7.517245e-01   
323.525000  47    -35.172904  0.844552 -34.328352 -35.187504  8.338244e-01   
347.300000  48    -35.246604  0.929629 -34.316975 -35.261804  9.205928e-01   
371.075000  49    -35.317691  1.018626 -34.2990

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')